In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import load_data
df = load_data.load_county_level()
df = df.sort_values(load_data.outcome_deaths, ascending=False)
print(df.shape)

(1212, 7306)
